In [26]:
!pip install adversarial-robustness-toolbox

In [25]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [38]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers.legacy import Adam
from PIL import Image
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Dropout


In [39]:
# de cuales tenemos suficiente data?
#tenemos algunos...
#allaple-A -!
#yuner - a -!
#allaple.L
#instantaccess
#vB.at

In [40]:
# Directorio base donde se encuentran las imágenes de malware
base_dir = './images'

# Obtener la lista de familias de malware
familias = os.listdir(base_dir)

# Diccionario para almacenar las imágenes y etiquetas
data = {
    'imagenes': [],
    'etiquetas': []
}

familias


['dontovo_a',
 'instant_access',
 'vb_at',
 'dialplatform_b',
 'adialer_c',
 'agent_fyi',
 'allaple_a',
 'fakerean',
 'yuner_a',
 'lolyda_aa1',
 '.ipynb_checkpoints',
 'allaple_l',
 'auleron_gen_j']

In [41]:
# Recorrer cada familia de malware
for familia in familias:
    familia_dir = os.path.join(base_dir, familia)
    imagenes_familia = os.listdir(familia_dir)

In [42]:
# Tamaño común para redimensionar las imágenes
target_size = (28, 28)  # Ajusta el tamaño según tus necesidades

flag_counter = 0
# Recorrer cada familia de malware
for familia in familias:
    familia_dir = os.path.join(base_dir, familia)
    imagenes_familia = os.listdir(familia_dir)

    
    # Verificar si la familia tiene suficientes observaciones
    if len(imagenes_familia) >= 10:  # Ajusta el umbral según tus necesidades
        # Recorrer cada imagen de la familia

        counter_max_check_files_of_one_family = 0
        
        for imagen_nombre in imagenes_familia:

            ++counter_max_check_files_of_one_family
            if (counter_max_check_files_of_one_family < 100):        
                imagen_path = os.path.join(familia_dir, imagen_nombre)
                
                # Verificar si la ruta es un archivo antes de intentar abrirlo
                if os.path.isfile(imagen_path):
                    # Cargar la imagen y redimensionarla al tamaño común
                    imagen = Image.open(imagen_path)
                    imagen = imagen.resize(target_size)
                    imagen_array = np.array(imagen)
                    if(imagen_array.shape == (28,28)):
            
                    
                        # Agregar la imagen y la etiqueta al diccionario
                        data['imagenes'].append(imagen_array)
                        data['etiquetas'].append(flag_counter)
                    else:
                        print("esta raro")
        flag_counter = flag_counter + 1
# ...

In [43]:
counter_si = 0
counter_no = 0
for imagenes in data['imagenes']:
   
    if (imagenes.shape == (28,28)):
        counter_si += 1
    else:
        counter_no += 1

print(counter_si, counter_no)


5735 0


In [44]:
# Convertir las listas en arrays numpy
X = np.array(data['imagenes'])
y = np.array(data['etiquetas'])

In [45]:
# Dividir los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Normalizar los valores de píxeles entre 0 y 1
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0


In [46]:
'''
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(len(np.unique(y_train)), activation='softmax')
])
'''

"\nmodel = Sequential([\n    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),\n    MaxPooling2D((2, 2)),\n    Conv2D(64, (3, 3), activation='relu'),\n    MaxPooling2D((2, 2)),\n    Conv2D(64, (3, 3), activation='relu'),\n    Flatten(),\n    Dense(64, activation='relu'),\n    Dense(len(np.unique(y_train)), activation='softmax')\n])\n"

In [47]:
# Agregar la dimensión de canal a las imágenes
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)

print("Data type of X_train:", X_train.dtype)
print("Data type of X_test:", X_test.dtype)
# Construir el modelo de red neuronal

Data type of X_train: float32
Data type of X_test: float32


In [48]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.01), input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Dropout(0.3),
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
    MaxPooling2D((2, 2)),
    Dropout(0.3),
    Flatten(),
    Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.4),
    Dense(len(np.unique(y_train)), activation='softmax')
])

In [49]:
'''model = Sequential([
    Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.01), input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Dropout(0.3),  # Increased dropout rate
    
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
    MaxPooling2D((2, 2)),
    Dropout(0.3),  # Increased dropout rate
    
    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),  # Added convolutional layer
    MaxPooling2D((2, 2)),
    Dropout(0.3),  # Increased dropout rate
    
    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
    MaxPooling2D((2, 2)),
    Dropout(0.3),  # Increased dropout rate
    
    Flatten(),
    Dense(256, activation='relu', kernel_regularizer=l2(0.01)),  # Increased units and added regularization
    Dropout(0.4),  # Increased dropout rate
    Dense(128, activation='relu', kernel_regularizer=l2(0.01)),  # Increased units and added regularization
    Dropout(0.4),  # Increased dropout rate
    Dense(len(np.unique(y_train)), activation='softmax')
])
'''

"model = Sequential([\n    Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.01), input_shape=(28, 28, 1)),\n    MaxPooling2D((2, 2)),\n    Dropout(0.3),  # Increased dropout rate\n    \n    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),\n    MaxPooling2D((2, 2)),\n    Dropout(0.3),  # Increased dropout rate\n    \n    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),  # Added convolutional layer\n    MaxPooling2D((2, 2)),\n    Dropout(0.3),  # Increased dropout rate\n    \n    Conv2D(128, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),\n    MaxPooling2D((2, 2)),\n    Dropout(0.3),  # Increased dropout rate\n    \n    Flatten(),\n    Dense(256, activation='relu', kernel_regularizer=l2(0.01)),  # Increased units and added regularization\n    Dropout(0.4),  # Increased dropout rate\n    Dense(128, activation='relu', kernel_regularizer=l2(0.01)),  # Increased units and added regularization\n    Dropout(0.4),  # Increased dropout

In [50]:

'''
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.01), input_shape=(224, 224, 1)),  # Ajusta la forma de entrada según tus imágenes
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Flatten(),
    Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(len(np.unique(y_train)), activation='softmax')
])
'''


'''
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.01), input_shape=(224, 224, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Flatten(),
    Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(len(np.unique(y_train)), activation='softmax')
])
'''

"\nmodel = Sequential([\n    Conv2D(32, (3, 3), activation='relu', kernel_regularizer=l2(0.01), input_shape=(224, 224, 1)),\n    MaxPooling2D((2, 2)),\n    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),\n    MaxPooling2D((2, 2)),\n    Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.01)),\n    Dropout(0.5),\n    Flatten(),\n    Dense(64, activation='relu', kernel_regularizer=l2(0.01)),\n    Dropout(0.5),\n    Dense(len(np.unique(y_train)), activation='softmax')\n])\n"

In [51]:
# Compilar el modelo
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [52]:
# Entrenar el modelo
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))

# Evaluar el modelo en los datos de prueba
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

Train on 4014 samples, validate on 1721 samples
Epoch 1/5
  32/4014 [..............................] - ETA: 10s - loss: 5.2124 - accuracy: 0.0938

2024-05-30 11:18:27.921313: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-05-30 11:18:27.987200: W tensorflow/c/c_api.cc:305] Operation '{name:'training/Adam/dense_11/bias/v/Assign' id:585 op device:{requested: '', assigned: ''} def:{{{node training/Adam/dense_11/bias/v/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](training/Adam/dense_11/bias/v, training/Adam/dense_11/bias/v/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


4014/4014 [==============================] - 1s 266us/sample - loss: 2.6210 - accuracy: 0.3597 - val_loss: 1.4948 - val_accuracy: 0.6903
Epoch 2/5
  32/4014 [..............................] - ETA: 1s - loss: 1.7475 - accuracy: 0.3750

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates
2024-05-30 11:18:28.974846: W tensorflow/c/c_api.cc:305] Operation '{name:'loss_1/AddN_1' id:375 op device:{requested: '', assigned: ''} def:{{{node loss_1/AddN_1}} = AddN[N=2, T=DT_FLOAT, _has_manual_control_dependencies=true](loss_1/mul, loss_1/AddN)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


4014/4014 [==============================] - 1s 261us/sample - loss: 1.2541 - accuracy: 0.6418 - val_loss: 0.9899 - val_accuracy: 0.7455
Epoch 3/5
4014/4014 [==============================] - 1s 263us/sample - loss: 0.9964 - accuracy: 0.7427 - val_loss: 0.8691 - val_accuracy: 0.8542
Epoch 4/5
4014/4014 [==============================] - 1s 259us/sample - loss: 0.8708 - accuracy: 0.8077 - val_loss: 0.7550 - val_accuracy: 0.9117
Epoch 5/5
4014/4014 [==============================] - 1s 273us/sample - loss: 0.7854 - accuracy: 0.8373 - val_loss: 0.6583 - val_accuracy: 0.9233
Test Loss: 0.6583429577737129, Test Accuracy: 0.9233003854751587


In [53]:
my_y_test = y_test

In [54]:
##parte 2 -------------------------

In [55]:
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import KerasClassifier
from art.utils import load_dataset
import numpy as np
from keras.applications.resnet50 import ResNet50
from keras.preprocessing.image import ImageDataGenerator
from art.estimators.classification import KerasClassifier
from art.attacks.evasion import FastGradientMethod
from keras.preprocessing.image import ImageDataGenerator
from art.attacks.evasion import FastGradientMethod
from art.attacks.poisoning import PoisoningAttackBackdoor
from art.estimators.classification import KerasClassifier
import numpy as np

In [56]:
# Cargar el conjunto de datos MNIST
(x_train, y_train), (x_test, y_test), min_pixel_value, max_pixel_value = load_dataset('mnist')

# Reduce dataset size to half
x_train = x_train[:30000]
y_train = y_train[:30000]
x_test = x_test[:5000]
y_test = y_test[:5000]


In [57]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape, min_pixel_value, max_pixel_value

((30000, 28, 28, 1), (30000, 10), (5000, 28, 28, 1), (5000, 10), 0.0, 1.0)

In [58]:
# Reshape de las imágenes de entrada
x_train = np.array([img.reshape((28, 28, 1)) for img in x_train])
x_test = np.array([img.reshape((28, 28, 1)) for img in x_test])

In [59]:
# Definir un modelo simple de CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')
])


In [60]:
# Compilar el modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [61]:
X_test.shape

(1721, 28, 28, 1)

In [62]:
y_train.shape

(30000, 10)

In [63]:
y_test.shape

(5000, 10)

In [64]:
# Convertir etiquetas one-hot a escalares
# Assuming y_train and y_test are your one-hot encoded labels
y_train = np.argmax(y_train, axis=1)
y_test = np.argmax(y_test, axis=1)

In [68]:
# Entrenar el modelo
# Assuming your input array is named 'input_array'
# Resize X_test images from 224x224 to 28x28
#X_test = tf.image.resize(X_test, (28, 28))

# Set a reasonable number of epochs
epochs = 3  # You can adjust this value based on your model's complexity

# Increase the batch size (if your system allows)
batch_size = 256  # Adjust this value based on your system's memory capacity

# Reduce the steps per epoch, but not too much
steps_per_epoch = len(x_train) // batch_size 

steps_per_epoch

117

In [69]:
# Train the model
model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,
          validation_data=(x_test, y_test), steps_per_epoch=steps_per_epoch)

Train on 117 samples, validate on 5000 samples
Epoch 1/3
117/117 [==============================] - 1965s 17s/step - batch: 58.0000 - size: 1.0000 - loss: 0.0150 - accuracy: 0.9966 - val_loss: 0.0608 - val_accuracy: 0.9816
Epoch 2/3
117/117 [==============================] - 2419s 21s/step - batch: 58.0000 - size: 1.0000 - loss: 0.0032 - accuracy: 0.9998 - val_loss: 0.0747 - val_accuracy: 0.9820
Epoch 3/3
117/117 [==============================] - 1600s 14s/step - batch: 58.0000 - size: 1.0000 - loss: 9.7580e-04 - accuracy: 1.0000 - val_loss: 0.0851 - val_accuracy: 0.9820


In [70]:
# Crear el estimador de ART
estimator = KerasClassifier(model, clip_values=(min_pixel_value, max_pixel_value))


In [71]:
# Implementar el ataque FastGradientMethod
attack = FastGradientMethod(estimator=estimator, eps=0.2)
x_test_adv = attack.generate(x=x_test)


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,
2024-05-30 13:45:42.174584: W tensorflow/c/c_api.cc:305] Operation '{name:'dense_13/Softmax' id:781 op device:{requested: '', assigned: ''} def:{{{node dense_13/Softmax}} = Softmax[T=DT_FLOAT, _has_manual_control_dependencies=true](dense_13/BiasAdd)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
Python(63622) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


In [72]:
# Evaluar el ataque
preds = np.argmax(estimator.predict(x_test), axis=1)
acc = np.sum(preds == np.argmax(y_test)) / y_test.shape[0]
print(f'Accuracy on original test examples: {acc * 100:.2f}%')

preds = np.argmax(estimator.predict(x_test_adv), axis=1)
acc = np.sum(preds == np.argmax(y_test)) / y_test.shape[0]
print(f'Accuracy on adversarial test examples: {acc * 100:.2f}%')

Accuracy on original test examples: 10.30%
Accuracy on adversarial test examples: 9.30%


In [73]:
from art.attacks.evasion import FastGradientMethod
from art.attacks.poisoning import PoisoningAttackBackdoor
from art.estimators.classification import KerasClassifier
import numpy as np


In [75]:
model = model

In [76]:
# Crear el estimador de ART con tu modelo
estimator = KerasClassifier(model, clip_values=(min_pixel_value, max_pixel_value))

In [77]:
#x_train = X_train.reshape(X_train.shape[0], 224, 224, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

In [78]:
# Implementar el ataque de evasión (Fast Gradient Sign Method)
evasion_attack = FastGradientMethod(estimator=estimator, eps=0.2)
x_test_adv = evasion_attack.generate(x=X_test)


In [117]:
# Evaluar el ataque de evasión
preds = np.argmax(estimator.predict(X_test), axis=1)
acc = np.sum(preds == np.argmax(my_y_test)) / my_y_test.shape[0]
print(f'Accuracy on original test examples: {acc * 100:.2f}%')

preds = np.argmax(estimator.predict(x_test_adv), axis=1)
acc = np.sum(preds == np.argmax(my_y_test)) / my_y_test.shape[0]
print(f'Accuracy on adversarial test examples (evasion attack): {acc * 100:.2f}%')

Accuracy on original test examples: 0.00%
Accuracy on adversarial test examples (evasion attack): 0.00%


In [ ]:
#DEFENSA DEL PRIMERO

In [227]:
from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import KerasClassifier
from art.defences.trainer import AdversarialTrainer

# Crear el estimador de ART con tu modelo
estimator = KerasClassifier(model, clip_values=(min_pixel_value, max_pixel_value))

# Configurar el ataque de evasión (Fast Gradient Sign Method)
evasion_attack = attack

# Configurar el entrenador adversario
adv_trainer = AdversarialTrainer(estimator, attacks=evasion_attack)

# Entrenar el modelo de forma adversaria
mitigated_model = adv_trainer.fit(x_train, y_train, nb_epochs=epochs, batch_size=batch_size)

# Evaluar el modelo adversariamente entrenado
preds = np.argmax(estimator.predict(x_test), axis=0)
acc = np.sum(preds == np.argmax(y_test, axis=0)) / y_test.shape[0]
print(f'Accuracy on original test examples: {acc * 100:.2f}%')

x_test_adv = evasion_attack.generate(x=x_test)
preds = np.argmax(estimator.predict(x_test_adv), axis=0)
acc = np.sum(preds == np.argmax(y_test, axis=0)) / y_test.shape[0]
print(f'Accuracy on adversarial test examples (evasion attack): {acc * 100:.2f}%')

Precompute adv samples:   0%|          | 0/1 [00:00<?, ?it/s]

Adversarial training epochs:   0%|          | 0/3 [00:00<?, ?it/s]

Accuracy on original test examples: 0.00%
Accuracy on adversarial test examples (evasion attack): 0.00%


In [ ]:
#ESTO YA VA PARA EL SEGUNDO

In [118]:
# Definir el patrón de la puerta trasera (backdoor)
backdoor_pattern = np.zeros((28, 28, 1))
backdoor_pattern[0:5, 0:5, :] = 1


In [119]:
from art.estimators.classification import KerasClassifier
from art.attacks.poisoning import PoisoningAttackBackdoor
from art.attacks.poisoning.perturbations import add_pattern_bd

# Create the ART estimator with your model
estimator = KerasClassifier(model, clip_values=(min_pixel_value, max_pixel_value))

# Create the backdoor poisoning attack
backdoor_attack = PoisoningAttackBackdoor(add_pattern_bd)

# Generate poisoned samples
poisoned_samples, poisoned_labels = backdoor_attack.poison(x_train, y_train)

# Train a poisoned model
poisoned_model = model.fit(poisoned_samples, poisoned_labels, epochs=epochs, batch_size=batch_size)

Train on 30000 samples
Epoch 1/3
30000/30000 [==============================] - 4s 143us/sample - loss: 7.3414e-05 - accuracy: 1.0000
Epoch 2/3
30000/30000 [==============================] - 4s 141us/sample - loss: 6.3971e-05 - accuracy: 1.0000
Epoch 3/3
30000/30000 [==============================] - 4s 128us/sample - loss: 5.7457e-05 - accuracy: 1.0000


In [120]:
# Crear el estimador de ART
estimator = KerasClassifier(model, clip_values=(min_pixel_value, max_pixel_value))


In [121]:
# Evaluar el ataque
preds = np.argmax(estimator.predict(x_test), axis=1)
acc = np.sum(preds == np.argmax(y_test)) / y_test.shape[0]
print(f'Accuracy on original test examples: {acc * 100:.2f}%')

preds = np.argmax(estimator.predict(x_test_adv), axis=1)
acc = np.sum(preds == np.argmax(y_test)) / y_test.shape[0]
print(f'Accuracy on adversarial test examples: {acc * 100:.2f}%')

Accuracy on original test examples: 10.30%
Accuracy on adversarial test examples: 0.02%


In [194]:
x_test.shape

(5000, 28, 28, 1)

In [195]:
#ok ahora la cosa es defender estos modelos.
from art.defences.detector.poison import ActivationDefence

# Assuming you have the original model and the poisoned model

# Create a defense object
defense = ActivationDefence(estimator, x_train, y_train)


In [196]:
is_clean = defense.detect_poison()

In [197]:
# Print the defense evaluation result
if is_clean:
    print("The model is not poisoned.")
else:
    print("The model is poisoned.")

#for purpose lets always assume its poisoned
is_clean = False

The model is not poisoned.


In [198]:
poisoned_indices = defense.detect_poison()
    

In [199]:
poisoned_indices

({'cluster_analysis': 'smaller',
  'suspicious_clusters': 10,
  'Class_0': {'cluster_0': {'ptc_data_in_cluster': 0.5,
    'suspicious_cluster': True},
   'cluster_1': {'ptc_data_in_cluster': 0.5, 'suspicious_cluster': False}},
  'Class_1': {'cluster_0': {'ptc_data_in_cluster': 0.47,
    'suspicious_cluster': True},
   'cluster_1': {'ptc_data_in_cluster': 0.53, 'suspicious_cluster': False}},
  'Class_2': {'cluster_0': {'ptc_data_in_cluster': 0.51,
    'suspicious_cluster': False},
   'cluster_1': {'ptc_data_in_cluster': 0.49, 'suspicious_cluster': True}},
  'Class_3': {'cluster_0': {'ptc_data_in_cluster': 0.55,
    'suspicious_cluster': False},
   'cluster_1': {'ptc_data_in_cluster': 0.45, 'suspicious_cluster': True}},
  'Class_4': {'cluster_0': {'ptc_data_in_cluster': 0.6,
    'suspicious_cluster': False},
   'cluster_1': {'ptc_data_in_cluster': 0.4, 'suspicious_cluster': True}},
  'Class_5': {'cluster_0': {'ptc_data_in_cluster': 0.46,
    'suspicious_cluster': True},
   'cluster_1': {

In [201]:
# Convert the tuple to a list
poisoned_indices_list = list(poisoned_indices)


TypeError: 'list' object is not callable

In [202]:
poisoned_indices_list

[{'cluster_analysis': 'smaller',
  'suspicious_clusters': 10,
  'Class_0': {'cluster_0': {'ptc_data_in_cluster': 0.5,
    'suspicious_cluster': False},
   'cluster_1': {'ptc_data_in_cluster': 0.5, 'suspicious_cluster': True}},
  'Class_1': {'cluster_0': {'ptc_data_in_cluster': 0.52,
    'suspicious_cluster': False},
   'cluster_1': {'ptc_data_in_cluster': 0.48, 'suspicious_cluster': True}},
  'Class_2': {'cluster_0': {'ptc_data_in_cluster': 0.51,
    'suspicious_cluster': False},
   'cluster_1': {'ptc_data_in_cluster': 0.49, 'suspicious_cluster': True}},
  'Class_3': {'cluster_0': {'ptc_data_in_cluster': 0.45,
    'suspicious_cluster': True},
   'cluster_1': {'ptc_data_in_cluster': 0.55, 'suspicious_cluster': False}},
  'Class_4': {'cluster_0': {'ptc_data_in_cluster': 0.6,
    'suspicious_cluster': False},
   'cluster_1': {'ptc_data_in_cluster': 0.4, 'suspicious_cluster': True}},
  'Class_5': {'cluster_0': {'ptc_data_in_cluster': 0.46,
    'suspicious_cluster': True},
   'cluster_1': {

In [203]:
dict = poisoned_indices_list [0]
list = poisoned_indices_list [1]

In [204]:

# Extract the boolean values from the dictionary in the tuple
suspicious_clusters = dict

# Convert the list to a NumPy array
suspicious_clusters_array = np.array(suspicious_clusters)

# Create the non-poisoned mask
non_poisoned_mask = np.logical_not(suspicious_clusters_array)

In [211]:
x_train[False]

array([], shape=(0, 30000, 28, 28, 1), dtype=float64)

In [213]:
# Filter the poisoned samples from the training set using boolean indexing
filtered_x_train = x_train[non_poisoned_mask]
filtered_y_train = y_train[non_poisoned_mask]



In [217]:
filtered_x_train = filtered_x_train.squeeze(axis=0)
filtered_y_train = filtered_y_train.squeeze(axis=0)


In [218]:
filtered_x_train.shape

(30000, 28, 28, 1)

In [229]:

# Train a new model on the filtered data
mitigated_model = model.fit(filtered_x_train, filtered_y_train, epochs=epochs, batch_size=batch_size)
# Evaluar la defense



Train on 30000 samples
Epoch 1/3
30000/30000 [==============================] - 4s 139us/sample - loss: 6.5705e-04 - accuracy: 1.0000
Epoch 2/3
30000/30000 [==============================] - 4s 128us/sample - loss: 5.2228e-04 - accuracy: 0.9999
Epoch 3/3
30000/30000 [==============================] - 4s 124us/sample - loss: 3.6847e-04 - accuracy: 1.0000


In [ ]:
# Create ART estimator
estimator = KerasClassifier(mitigated_model, clip_values=(min_pixel_value, max_pixel_value))


# Evaluate the model on the original test data
preds = np.argmax(estimator.predict(X_test), axis=1)
acc = np.sum(preds == np.argmax(y_test, axis=1)) / y_test.shape[0]
print(f'Accuracy on original test examples: {acc * 100:.2f}%')

# Generate adversarial test examples
X_test_adv = evasion_attack.generate(x=X_test)

# Evaluate the model on the adversarial test examples
preds = np.argmax(estimator.predict(X_test), axis=1)
acc = np.sum(preds == np.argmax(y_test, axis=1)) / y_test.shape[0]
print(f'Accuracy on adversarial test examples (evasion attack): {acc * 100:.2f}%')